In [15]:
import cv2
import numpy as np
keep_percent = 0.75
cnv = 1.4
thresh = 125.0
nfeat = 2**15 - 4
# Load the images

def read_img(n):
    s1 = '2023_09_01_SonyRX1RM2_g201b20538_f001_'
    s3 = '.JPG'
    s = str(n)
    l = len(s)
    s2 = '0' * (4-l) + s
    return cv2.imread('./folder/' + s1 + s2 + s3)

def crop_img(image_1):
    mask = np.any(image_1 != 0, axis=-1)
    coords = np.argwhere(mask)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0) + 1
    return image_1[y_min:y_max, x_min:x_max]
    


def image_intersect(image1, image2):
    height, width, _ = image1.shape
    canvas_height = round(height * cnv)
    canvas_width = round(width * cnv)
    green_canvas = np.zeros((canvas_height, canvas_width, 4), dtype=np.uint8)
    green_canvas[:] = (0, 0, 0, 0)  # Set the canvas to green
    y_offset = (canvas_height - height) // 2
    x_offset = (canvas_width - width) // 2
    alpha_image1 = np.zeros((height, width, 4), dtype=np.uint8)
    if(image1.shape[2] == 3):
        alpha_image1[:,:,0:3] = image1
        alpha_image1[:,:,3] = 255
    else:
        alpha_image1 = image1
    green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = alpha_image1
    image1 = green_canvas

    height, width, _ = image2.shape
    canvas_height = round(height * cnv)
    canvas_width = round(width * cnv)
    green_canvas = np.zeros((canvas_height, canvas_width, 4), dtype=np.uint8)
    green_canvas[:] = (0, 0, 0, 0)  # Set the canvas to green
    y_offset = (canvas_height - height) // 2
    x_offset = (canvas_width - width) // 2
    alpha_image2 = np.zeros((height, width, 4), dtype=np.uint8)
    if(image2.shape[2] == 3):
        alpha_image2[:,:,0:3] = image2
        alpha_image2[:,:,3] = 255
    else:
        alpha_image2 = image2
    green_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = alpha_image2
    image2 = green_canvas

    
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(nfeatures = nfeat)
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
 
    # Match descriptors.
    matches = bf.match(descriptors1, descriptors2)
    
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    keep = int(len(matches) * keep_percent)
    matches = matches[:keep]
    

    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    print('bijection part of transform: ', len(matches), '/', nfeat, sep = '')

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find the rotation transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(points2, points1, method=cv2.RANSAC, ransacReprojThreshold = thresh)
    print('inner part of transform: ', np.count_nonzero(_ == 1), '/', nfeat, sep = '')
    print(matrix)
    s = matrix[0, 1]
    c = matrix[0, 0]
    f = np.arctan(s/c)
    print('angle is', f)
    print('scale is', c/np.cos(f))
    
    # Apply the rotation transformation
    result = cv2.warpAffine(image2, matrix, (image1.shape[1], image1.shape[0]))
    
    h, w, _ = result.shape

    # Define the region of interest (ROI) on the base image
    roi = image1[0:h, 0:w]

    # Create a mask of the overlay image and its inverse mask
    result_gray = cv2.cvtColor(result, cv2.COLOR_BGRA2GRAY)
    _, mask = cv2.threshold(result_gray, 1, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Black-out the area of the overlay in the ROI
    image1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

    # Take only the region of the overlay from the overlay image
    result_fg = cv2.bitwise_and(result, result, mask=mask)
    
    # Put the overlay in the ROI and modify the main image
    dst = cv2.add(image1_bg, result_fg)
    image1[0:h, 0:w] = dst
    
    mask = np.any(image1 != 0, axis=-1)

    cropped_image = crop_img(image1)

    # Save the cropped image
    cv2.imwrite('cropped_image.png', cropped_image)
    print('assembled one image')
    
    return cropped_image

image = read_img(3)
for i in range (4, 5):
    imagep = read_img(i)
    image = image_intersect(image, imagep)
    print(i)


bijection part of transform: 7350/32764
inner part of transform: 2440/32764
[[ 9.62433635e-01  1.93252858e-01 -8.55647009e+02]
 [-1.93252858e-01  9.62433635e-01  8.14314183e+02]]
angle is 0.1981608612780551
scale is 0.9816441149525381
11133 7426
assembled one image
4


In [8]:
a = [[1,2],[3,4]]
b = [[1, 1],[2,3]]
print(np.dot(a, b))

[[ 5  7]
 [11 15]]


./folder/2023_09_01_SonyRX1RM2_g201b20538_f001_2345.JPG
